In [ ]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
import geojson
from numpy import save, load
import os.path
import sys
from geojson import Feature, FeatureCollection
import json
import geojson
from shapely.geometry import shape
from shapely.geometry.point import Point
from shapely.geometry.polygon import Polygon

In [ ]:
####################### STEP 1: GATHERING DATA #####################################

# this is a map with polygons for each region, source: https://github.com/electricitymap/electricitymap-contrib/blob/919f0b0d9f93eaa7a862a036fc3877932bac37cf/web/geo/world.geojson
path=r'world.geojson'
with open(path,'r') as data_file:
    data1 = json.load(data_file)
    
# gather all pplygons in list
feature_collection = FeatureCollection(data1['features'])


# this data holds the average co2 emission per zone of 2021, source partially https://github.com/electricitymap/ and partially https://github.com/Breakend/experiment-impact-tracker
path=r'Data/co2_emission_per_zone.json'
with open(path,'r') as data_file:
    data = json.load(data_file)

# there are a few zones that we don't have information about, we call these "wrong_zones" 
# and eliminate them from our world map
polygons=[]
zone_names=[]
wrong_zones = []

for feature in feature_collection["features"]:
    poly: Polygon = shape(feature['geometry'])
    if(feature['properties']['zoneName'] in data):
        polygons.append(poly)
        zone_names.append([feature['properties']['zoneName'], feature['properties']['countryKey'], feature['properties']['countryName']])
    else: 
        if(feature['properties']['zoneName'] not in wrong_zones): 
            wrong_zones.append(feature['properties']['zoneName'])
        #print(feature['properties']['zoneName'])

# function to later test if data point is actually in world (in a polygon) or if it is water/sea
def isInAPolygon(x,y):
    for m in range(len(polygons)):
        if(polygons[m].contains(Point(x,y))):
            return True   
    
# Get GPUs (needed for their tdp_watts value)
gpus = []
with open("Data/gpus.csv","r",encoding="Latin1") as gpus1:
    tempgpus = csv.reader(gpus1)
    for row in tempgpus:
        # doing this because otherwise the first row with the column titles would also be added
        if(row[0] != 'name'):
            gpus.append([row[0], row[2]])
df_gpus = pd.DataFrame(gpus, columns = ['name', 'tdp_watts'])
df_gpus.tdp_watts = pd.to_numeric(df_gpus.tdp_watts, errors = 'coerce')

print(zone_names[0:10])
print(data)
print(df_gpus.head())

In [ ]:
########################## STEP 2: PREPARING DATA FOR TRAINING ########################################
# we take 1000*500 locations in the world and get the emission based on which region they are in
# later, we will use these sample locations as input for the ML model
training_points = []
fortschritt = 0
x = np.linspace(-180,180,750)
y =  np.linspace(-90,90,350)

for j in range(len(y)):
    for i in range(len(x)):
        point = Point(x[i], y[j])
        poly_nr = -1
        # check if point is in a polygon, if yes, get the zone name of that polygon and append to training data
        for m in range(len(polygons)):
            if(polygons[m].contains(point) and not zone_names[m] in wrong_zones):
                poly_nr = m
                break
        if(poly_nr != -1):
            training_points.append([[x[i],y[j]], zone_names[poly_nr]])
        if(fortschritt % 10000 == 0): print(fortschritt)
        fortschritt +=1
    
# split data in X and y and also in train and test data
X = [x[0] for x in training_points]
y = [data[t[1][0]]['hydro discharge']['value'] for t in training_points]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)

In [ ]:
################# STEP 3: TRAIN ML MODEL ##########################################
# we chose a RandomForest Regression Model

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=30, random_state=100)
# train model
regr.fit(X_train,y_train)
# test model
y_pred = regr.predict(X_test)
# print r2 score
score = r2_score(y_test, y_pred)
print(f"Test Score is: {score}! Everything above 0.8 is good!")

In [ ]:
# now we do the same for other models
# we try a SVM Regression Model
regr = svm.SVR()
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"Test Score is: {score}! Everything above 0.8 is good!")

In [ ]:
# we try a Linear Regression Model
regr = LinearRegression()
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"R2 Score is {score}, everything above 0.8 is good")

In [ ]:
from sklearn import tree
regr = tree.DecisionTreeRegressor()
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"Test Score is: {score}! Everything above 0.8 is good!")

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=77, max_iter=1000)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"R2 Score is {score}, everything above 0.8 is good")

In [ ]:
################################ STEP 4: PLOT DATA #####################################
# we plot the world map, first only the polygons, then the training data, and then the predicted data

# plot polygons
fig = plt.figure(figsize=(15, 8))
for pol in polygons:
    if pol.geom_type == 'MultiPolygon':
        for g in pol.geoms:
            x,y = g.exterior.xy
            plt.plot(x,y)
    elif pol.geom_type == 'Polygon':
        x,y = pol.exterior.xy
        plt.plot(x,y)
plt.show()
plt.close(fig)

# plot training data
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(1,1,1)
# iterate over hydro discharge values to get max and min values
maximp, minimp = 0,1000
for d in data:
    if(data[d]['hydro discharge']['value'] > maximp): maximp = data[d]['hydro discharge']['value']
    if(data[d]['hydro discharge']['value'] < minimp): minimp = data[d]['hydro discharge']['value']
normal_imps = []
col = []
xs = []
ys = []
for t in training_points:
    imp = data[t[1][0]]['hydro discharge']['value']
    n_imp = 1 - (float(imp-minimp)/maximp-minimp)
    normal_imps.append(n_imp)
    col.append(plt.cm.hot(n_imp))
    xs.append(t[0][0])
    ys.append(t[0][1])
ax.scatter(xs,ys,c=col,s=0.1)
ylim = ax.get_ylim()
xlim = ax.get_xlim()
plt.show()
plt.close(fig)


# plot predicted world emission map
x = np.linspace(-180,180,400)
y =  np.linspace(-90,90,400)
X, Y = np.meshgrid(x,y)

xyvals = []
for j in range(len(y)):
    for i in range(len(x)):
        xyvals.append([x[i], y[j]])
T = regr.predict(xyvals)
print(T)
fortschritt = 0
normal_T = []
#set impact where there is no data to 0
for i in range(len(T)):
    if (not isInAPolygon(xyvals[i][0],xyvals[i][1])):
        T[i] = 0
    if(fortschritt % 10000 == 0): print(fortschritt)
    fortschritt +=1
print(T)

#normalize the impact to get values between 0 and 1
max_T = max(T)
min_T = min(T)
print(max_T)
print(min_T)
normal_T = [1 - ((float(i)- min_T)/(max_T-min_T)) for i in T]
print("i am here")

#add a mask that is true where there is no data (otherwise it wont be white in the plot)
normal_T= np.ma.masked_array(normal_T,mask=[t==1 for t in normal_T])

#reshape the impact array s.t. it fits the X and Y shape
normal_T = normal_T.reshape(-1,400)

#create a filled contour plot with the coordinates and the impact
fig = plt.figure(figsize=(15, 8))
ax = fig.add_subplot(1,1,1)
ax.set_facecolor("white")
ax.contourf(X, Y, normal_T, cmap=plt.cm.hot)
ax.set_ylim(ylim)
ax.set_xlim(xlim)
plt.show()

In [ ]:
def get_predicted_value(hours, gpu_name, location_string):
    #result = df_impact.loc[(df_impact_old['provider'] == provider) & (df_impact_old['region'] == region) & (df_impact_old['name'] == gpu_name)].iloc[0]
    #result = df_impact.loc[(df_impact_old['name'] == gpu_name)]
    app = Nominatim(user_agent='mina')
    location = app.geocode(location_string).raw
    impact = regr.predict([[location['lon'], location['lat']]])
    return hours * compute_impact(gpu_name, hours, impact[0])

import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from IPython.core.display import HTML
from IPython.display import display,clear_output


hardware_types = ["A100 PCIe 40/80GB"," A100 SXM4 80 GB"," AGX Xavier"," AMD EPYC 7763",
        " AMD RX480","GTX 1080","GTX 1080 Ti"," GTX 750","GTX TITAN X",
        "Intel Xeon E5-2630v4","Intel Xeon E5-2650","Intel Xeon E5-2699",
        "Intel Xeon Gold 5220","Intel Xeon Gold 6148","Quadro K6000",
        "Quadro P6000","RTX 2080","RTX 2080 Ti","RTX 3080"," RTX 3080 TI",
        " RTX 3090"," RTX 8000","RTX A6000","T4","TITAN X Pascal","TPUv2 Chip",
        "TPUv3 Chip","Tesla K40c","Tesla K80","Tesla M40 24GB","Tesla P100",
        "Tesla P40","Tesla V100-PCIE-16GB","Tesla V100-SXM2-16GB","Tesla V100-SXM2-32GB",
        "Titan RTX","Titan V","Titan Xp"
       ]

hardware_type_dropdown = widgets.Dropdown(
    options=hardware_types,
    value=hardware_types[0],
    description='Hardware:',
    disabled=False,
    layout={'width': '270px'}
)

city = widgets.Text(
    value='Wuhan',
    placeholder='Enter City!',
    description='City:',
    disabled=False,
    layout={'width': '270px'}
)
country = widgets.Text(
    value='China',
    placeholder='Enter Country!',
    description='Country:',
    disabled=False,
    layout={'width': '270px'}
)

floatButton = widgets.BoundedFloatText(
    value=100,
    min=0,
    max=1000.0,
    step=0.5,
    description='Hours Used:',
    disabled=False,
    layout={'width': '270px'}
)

button = widgets.Button(
    description='Compute',
    disabled=False,
    button_style='danger', 
    tooltip='Compute CO2 emission',
    icon='check',
)


items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    width='60%'
                    )
box_layout2 = Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%'
                    )
box_layout3 = Layout(display='flex',
                    flex_flow='column',
                     align_items="center",
                    height='150px',
                    width = '100%',
                    )

items = [hardware_type_dropdown, floatButton]
items2 = [city, country]

items3 = [button]
display(HTML('<h1 style = "text-align: center;color:red;font-family: fantasy;font-size: 50px;"><b><u>CO2 Emission Calculator</u></<b></h1>'))

box = Box(children=items, layout=box_layout)

box2 = Box(children=items2, layout=box_layout)


box3 = Box(children=items3, layout=box_layout2)

items4 = [box, box2, box3]
box4 = Box(children=items4, layout=box_layout3)
display(box4)

html_code ="""
  <h3 style = "text-align: center; color:blue;font-style:italic;font-weight:bold" id="result_label"><b></<b></h5>"""
display(HTML(html_code))

def on_button_clicked(b):
    global i
    i+=1
    with output:
        clear_output(wait=True)
        city_name = city.value
        country_name = country.value
        place = city_name + ', '+country_name
    
        predicted_val = get_predicted_value(floatButton.value, str(hardware_type_dropdown.value),place)

        html_code2 ="""
<html>
  <head>
    <script type="text/javascript">
    document.getElementById("result_label").innerText = "Predicted Value for CO2 Emission is %s";
    </script>
  </head>
</html>""" % round(predicted_val,2)
        display(HTML(html_code2))
    
i=0
output = widgets.Output()
display(output)
button.on_click(on_button_clicked)

box.add_class("box_style2")
box2.add_class("box_style2")
button.add_class("button_style")
 

HTML("""
<style>
.button_style{
    float: center;
}
.jp-Notebook {
font-family: Arial, Helvetica, sans-serif;
}
.widget-inline-hbox .widget-label {
font-family: cursive;
font-weight: bold;
font-size: 16px;
width: 100px;
}
.button_style {
width: 301px;
height: 33px;
font-size: 18px;
font-weight: bold;
}
</style>
""")